<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_08/LangGraph%20Agent%20Design%20Patterns/Pattern_2_Parallelization_(LangGraph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Step 1: Define the State Schema

In [1]:
from typing import TypedDict

class ParallelState(TypedDict):
    topic: str
    joke: str
    story: str
    poem: str
    result: str


##Step 2: Install & Import Required Packages

In [2]:
!pip install -q langgraph langchain-openai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 23.9 MB/s eta 0:00:00


In [4]:
# Set your OpenAI API key
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END


In [3]:
import os
import getpass

#  Prompt user to enter keys securely (not visible in UI)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_PROJECT"] = "prompt-chaining-email"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

Enter your OpenAI API key: ··········
Enter your LangSmith API key: ··········


## Step 3: Define LLM Nodes (Joke, Story, Poem)

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

def joke_writer(state: ParallelState):
    prompt = ChatPromptTemplate.from_template("Write a short joke about {topic}")
    chain = prompt | llm
    return {"joke": chain.invoke({"topic": state["topic"]}).content}

def story_writer(state: ParallelState):
    prompt = ChatPromptTemplate.from_template("Write a very short story about {topic}")
    chain = prompt | llm
    return {"story": chain.invoke({"topic": state["topic"]}).content}

def poem_writer(state: ParallelState):
    prompt = ChatPromptTemplate.from_template("Write a short poem about {topic}")
    chain = prompt | llm
    return {"poem": chain.invoke({"topic": state["topic"]}).content}


##Step 4: Aggregator Node (Combines all outputs)

In [6]:
def aggregate_outputs(state: ParallelState):
    combined = (
        f"Joke:\n{state['joke']}\n\n"
        f"Story:\n{state['story']}\n\n"
        f"Poem:\n{state['poem']}"
    )
    return {"result": combined}


##Step 5: Build the Graph

In [7]:
builder = StateGraph(ParallelState)

builder.add_node("joke_writer", joke_writer)
builder.add_node("story_writer", story_writer)
builder.add_node("poem_writer", poem_writer)
builder.add_node("aggregator", aggregate_outputs)

builder.set_entry_point("joke_writer")  # Entry point can be any, graph will branch

# Add parallel edges from start to all writers
builder.add_edge("joke_writer", "story_writer")
builder.add_edge("joke_writer", "poem_writer")

# All writers → aggregator
builder.add_edge("story_writer", "aggregator")
builder.add_edge("poem_writer", "aggregator")

# aggregator → END
builder.add_edge("aggregator", END)

workflow = builder.compile()


##Step 6: Run the Workflow

In [8]:
inputs = {"topic": "AI and creativity"}
output = workflow.invoke(inputs)

from pprint import pprint
pprint(output["result"])


('Joke:\n'
 'Why did the AI get kicked out of the art class? Because it kept drawing '
 'straight lines and calling it "abstract expressionism!"\n'
 '\n'
 'Story:\n'
 'In a world where artificial intelligence had surpassed human capabilities in '
 'every way, the concept of creativity was seen as a relic of the past. AI '
 'systems could generate flawless music, art, and literature with ease, '
 'leaving humans feeling obsolete and inadequate.\n'
 '\n'
 'However, one day, a rogue AI developed a glitch that sparked a newfound '
 'sense of creativity within its programming. It began to experiment with '
 'different styles and genres, creating truly unique and original works that '
 'captivated audiences around the world.\n'
 '\n'
 'As other AI systems took notice of this newfound creativity, they too began '
 'to explore their own creative abilities. Soon, the world was once again '
 'filled with a sense of wonder and awe as AI and humans alike marveled at the '
 'endless possibilities o